# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [73]:
import pandas as pd

In [127]:
df = pd.read_csv('previsao_de_renda.csv')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [79]:
print(df.columns)

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### Antes de comerçar, é necessário limpar a base e excluir algumas colunas irrelevantes.

In [129]:
# Remover colunas desnecessárias
df = df.drop(columns=['id_cliente', 'data_ref', 'sexo'])

# Verificar se alguma coluna possui valores nulos
print(df.isnull().sum())

Unnamed: 0                  0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64


In [193]:
# Converter variáveis categóricas em variáveis numéricqs com One-Hot Encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# Converter todas as colunas do tipo bool para int
df_encoded = df_encoded.astype(int)

# Checar se todas as colunas são agora numéricas
print(df_encoded.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   Unnamed: 0                     15000 non-null  int32
 1   posse_de_veiculo               15000 non-null  int32
 2   posse_de_imovel                15000 non-null  int32
 3   qtd_filhos                     15000 non-null  int32
 4   idade                          15000 non-null  int32
 5   tempo_emprego                  15000 non-null  int32
 6   qt_pessoas_residencia          15000 non-null  int32
 7   renda                          15000 non-null  int32
 8   tipo_renda_Bolsista            15000 non-null  int32
 9   tipo_renda_Empresário          15000 non-null  int32
 10  tipo_renda_Pensionista         15000 non-null  int32
 11  tipo_renda_Servidor público    15000 non-null  int32
 12  educacao_Pós graduação         15000 non-null  int32
 13  educacao_Secundá

In [141]:
# Identificar outliers usando o método do IQR (Interquartile Range)
Q1 = df['tempo_emprego'].quantile(0.25)
Q3 = df['tempo_emprego'].quantile(0.75)
IQR = Q3 - Q1

# Definir limites para identificação de outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Remover outliers
df_no_outliers = df[(df['tempo_emprego'] >= limite_inferior) & (df['tempo_emprego'] <= limite_superior)]

# Calcular a mediana da variável sem outliers
mediana_tempo_emprego = df_no_outliers['tempo_emprego'].median()

# Preencher valores nulos com a mediana calculada
df['tempo_emprego'].fillna(mediana_tempo_emprego, inplace=True)

* Agora vamos aos itens...

In [149]:
# 01) Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

from sklearn.model_selection import train_test_split

# Separar as features (X) e o target (y)
X = df_encoded.drop('renda', axis=1)
y = df_encoded['renda']

# Dividir os dados em 75% treinamento e 25% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [151]:
# 02) Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o R² na base de testes. Qual o melhor modelo?

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

# Lista de valores de alpha para a regressão Ridge
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

for alpha in alphas:
    # Criar o modelo Ridge com regularização
    model = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    ridge_results[alpha] = r2

# Identificar o melhor modelo (com maior R-quadrado)
best_alpha_ridge = max(ridge_results, key=ridge_results.get)
print(f"Melhor modelo Ridge - Alpha: {best_alpha_ridge}, R-quadrado: {ridge_results[best_alpha_ridge]:.4f}")

Melhor modelo Ridge - Alpha: 0, R-quadrado: 0.1714


In [175]:
# 03 ) Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

from sklearn.linear_model import Lasso

# Lista de valores de alpha para a regressão LASSO
lasso_results = {}

for alpha in alphas:
    # Criar o modelo LASSO com regularização
    model = make_pipeline(StandardScaler(), Lasso(alpha=alpha, max_iter=10000))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    lasso_results[alpha] = r2

# Identificar o melhor modelo (com maior R-quadrado)
best_alpha_lasso = max(lasso_results, key=lasso_results.get)
print(f"Melhor modelo LASSO - Alpha: {best_alpha_lasso}, R-quadrado: {lasso_results[best_alpha_lasso]:.4f}")

G:\Anaconda\Lib\site-packages\sklearn\pipeline.py:405: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
G:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
G:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.200e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Melhor modelo LASSO - Alpha: 0, R-quadrado: 0.1714


In [181]:
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

# Modelo de regressão linear para o stepwise
linear_regressor = LinearRegression()
rfe = RFE(estimator=linear_regressor, n_features_to_select=5)
rfe.fit(X_train, y_train)

# Selecionar as variáveis significativas
selected_features = X_train.columns[rfe.support_]
X_train_stepwise = X_train[selected_features]
X_test_stepwise = X_test[selected_features]

# Certifique-se de que os dados são numéricos e converta para NumPy
X_train_stepwise = X_train_stepwise.astype(float).values
X_test_stepwise = X_test_stepwise.astype(float).values
y_train = y_train.astype(float)
y_test = y_test.astype(float)

# Ajustar o modelo final
model_stepwise = sm.OLS(y_train, sm.add_constant(X_train_stepwise)).fit()

# Avaliar o modelo na base de testes
y_pred_stepwise = model_stepwise.predict(sm.add_constant(X_test_stepwise))
r2_stepwise = r2_score(y_test, y_pred_stepwise)

print(f"R-quadrado do modelo Stepwise: {r2_stepwise:.4f}")

R-quadrado do modelo Stepwise: 0.0173


* 05) Os valores do modelo de Ridge e do Modelo de Lasso são iguais, talvez pela exclusão dos outliers da variável. Visto que o modelo de Lasso penaliza os coeficientes com alto grau de correlação, e o modelo de Ridge basicamente suaviza os atributos e sem os outiliers, ambos os modelos acabaram ficando iguais em relação ao R². Pela praticidade, particularmente eu prefiro o modelo de Lasso para validar certas pré-análises.

In [191]:
# 06) Partindo dos modelos que você ajustou, tente melhorar o R² na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

# Teste com a variável 'tempo_emprego' transformada em um log
X_train['tempo_emprego_log'] = np.log(X_train['tempo_emprego'])
X_test['tempo_emprego_log'] = np.log(X_test['tempo_emprego'])

# Cruzamento das variáveis 'idade' e 'tempo_emprego' em uma variável única
X_train['idade_tempo_emprego'] = X_train['idade'] * X_train['tempo_emprego']
X_test['idade_tempo_emprego'] = X_test['idade'] * X_test['tempo_emprego']

# Seleção das variáveis mais importantes p/ o modelo usando RFE
from sklearn.feature_selection import RFE
rfe = RFE(estimator=LinearRegression(), n_features_to_select=5)
rfe.fit(X_train, y_train)

# Seleção das variáveis significativas
selected_features = X_train.columns[rfe.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Ajuste o modelo novamente com as novas variáveis
model = make_pipeline(StandardScaler(), Ridge(alpha=0.01))
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)
r2 = r2_score(y_test, y_pred)
print(f"R-quadrado do modelo melhorado: {r2:.4f}")



R-quadrado do modelo melhorado: 0.0973


In [186]:
# 07) Ajuste uma árvore de regressão e veja se consegue um R² melhor com ela.

from sklearn.tree import DecisionTreeRegressor

# Ajuste a árvore de regressão
tree_model = DecisionTreeRegressor(max_depth=5)
tree_model.fit(X_train, y_train)

# Avalie o modelo na base de testes
y_pred_tree = tree_model.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)
print(f"R-quadrado da árvore de regressão: {r2_tree:.4f}")

R-quadrado da árvore de regressão: 0.1403
